Compute the STAs of all the moth campaniform sensilla. Then, use it to build a covariance matrix

In [1]:
import numpy as np
import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt
import h5py

In [31]:
data_path = 'data/V1_Ringach/'

# load stimuli used to construct the STAs
data = loadmat(data_path + 'db_1.mat')['db']

In [47]:
len(data[0, 0])

12